# Проекция. Франк-Вульф 

## Основная часть

Рассмотрим задачу минимизации эмпирического риска:

\begin{equation}
\min_{x \in \mathcal{X} \subset \mathbb{R}^d} \left[ f(x) = \frac{1}{n} \sum\limits_{i=1}^n \ell_x(a_i, b_i)\right],
\end{equation}

где:
- $\ell_x(a_i, b_i)$ — функция потерь (cross-entropy loss),
- $x$ — вектор параметров модели,
- $\{a_i, b_i\}_{i=1}^n$ — выборка данных.

Функция потерь для каждого объекта $i$ записывается как:

\begin{equation}
\ell_x(a_i, b_i) = -b_i \ln(p(x^Ta_i)) - (1 - b_i) \ln(1 - p(x^Ta_i)),
\end{equation}

где $p(x^Ta_i)$ — это вероятность, вычисляемая с помощью логистической функции в комбинации с линейной моделью:

\begin{equation}
p(x^Ta_i) = \frac{1}{1 + \exp(-x^T a_i)}.
\end{equation}

Градиент для нашей целевой функции:
\begin{equation}
\nabla f(x) = \frac{1}{n} \sum_{i=1}^n (p(x^Ta_i) - b_i) a_i
\end{equation}
В качестве константы Липшица нашей целевой функции можно брать
\begin{equation}
L = \frac{1}{4n} \lambda_{\max}(A A^T)
\end{equation}

К заданию приложен датасет _mushrooms_. С помощью следующего кода сформируем матрицу $A$ и вектор $b$, в которой и будет храниться выборка $\{a_i, b_i\}_{i=1}^n$:

In [1]:
#файл должен лежать в той же деректории, что и notebook
dataset = "mushrooms.txt"

from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
A, b = data[0].toarray(), data[1]

b = b-1

Разделим данные на две части: обучающую и тестовую.

__Важно:__ обязательно дальше при решении задания для обучения используйте train выборку, а для проверки test.

In [2]:
from sklearn.model_selection import train_test_split
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=42)

Зафиксируем seed для воспроизводимости. Для генерации случайных точек используйте созданный генератор `rng` ([документация](https://numpy.org/doc/stable/reference/random/generator.html)).

In [3]:
import numpy as np

seed = 42
rng = np.random.default_rng(seed)

Для обучающей части $A_{train}$, $b_{train}$ оцените константу $L$.

Можно просто вставить реализацию из прошлого дз.

In [5]:
# Ваше решение
n = A_train.shape[0]

eigvals = np.linalg.eigvals(A_train.T @ (A_train)) 
lambda_max = (np.max(eigvals) / (4 * n)).real
L = lambda_max # с точностью до 1/1000 
lmbd = L / 1000
mu = lmbd
print(f"L = {L}")
print(f"mu = {mu}")

L = 2.5846388628723007
mu = 0.0025846388628723007


Реализуйте в коде подсчет значения и градиента для нашей целевой функции. При этом $A$, $b$, необходимо подавать в качестве параметра, чтобы была возможность их менять.

Необходимо использовать только библиотеку ``numpy``.

Можно просто вставить немного модифицированную реализацию из прошлого дз.

In [7]:
# Ваше решение
def p_func(z):
    return 1.0 / (1.0 + np.exp(-z))

def f_func(x, A, b, lam=lmbd, eps=1e-10):
    n = A.shape[0]
    p = p_func(A @ x)
    return -np.mean(b * np.log(p + eps) + (1 - b) * np.log(1 - p + eps))

def grad_func(x, A, b, lam=lmbd, eps=1e-10):
    n = A.shape[0]
    p = p_func(A @ x)
    return (1 / n) * A.T.dot(p - b)

def criterion(x, A, b, lam=lmbd):
    return np.linalg.norm(grad_func(x, A, b, lam)) / np.linalg.norm(grad_func(x_0, A, b, lam))

__Задача 1. (всего 5 баллов)__ Так как мы теперь решаем задачу оптимизации на шаре, необходимы методы, учитывающие это.

__а). (2 балла)__ Докажите, что для $\ell_1$-шара с центром в $0$ и радиусом $R$  выражение для решения задачи линейной оптимизации при заданном векторе $g \in \mathbb{R}^d$:
$$
s^* = \arg \min_{s \in \mathcal{X}} \langle s, g \rangle.
$$
может быть выписано так:
$$
s^*: \begin{cases}
s^*_j = -R \cdot \mathrm{sign}(g_j), &j = \min\{\arg\max\limits_{i}|g_i|\}, \\ s^*_k = 0, &k \neq j \end{cases}
$$

Формально обоснуйте свой ответ, например, можно (необязательно именно так) использовать условия ККТ.


**Ваше решение**

Целевая функция является линейной, поэтому её минимум на шаре достигается в одной из точек, у которых одна координата равна $\pm R$, а остальные 0. Чтобы минимизировать выражение, найдём координату $g$ с максимальным по модулю значением ($\arg\max\limits_{i}|g_i|$), и возьмём $s^*_j = -R \cdot \mathrm{sign}(g_j)$ (остальные индексы равны 0), ведь это и даст минимальное значение. В определении $j$ стоит `min` для учёта случая, когда индексов с максимальным по модулю значением несколько.

Для проверки и обоснования выпишем лагранжиан:

$$
\mathcal{L}(s,\lambda) = \sum_{i=1}^{d} s_i g_i + \lambda \left( \sum_{i=1}^{d} |s_i| - R \right)
$$

Условия ККТ в порядке 2-1-3:

Доп. нежёсткость выполняется, т.к мы выбрали такую точку, что при любом $\lambda$ второе слагаемое лагранжиана равно 0

Условие стационарности можно записать в виде равенства 0 производной: $g_i + \lambda \cdot sign(s_i) = 0$ для любого $i$. Для того индекса $j$, который нашли из $argmax$, получим отсюда $\lambda = |g_j|$. Для остальных мы взяли $s_i = 0$, то есть функция недифф., поэтому необходимо, чтобы $g_i$ попал в субдифф. функции $|s_i|$, равный отрезку от $-\lambda$ до $\lambda$. Поскольку $\lambda$ - максимальное по модулю значение $g_i$, все остальные координаты $g$ попадут в этот отрезок.

Как было найдено, $\lambda = |g_j| > 0$


__б). (1 балл)__ Реализуйте отдельно решение задачи линейной оптимизации из предыдущего пункта (радиус шара $R$ лучше передавать в качестве параметра). Реализуйте метод Франк-Вульфа для нашей задачи.

**Псевдокод алгоритма**

*Инициализация:*

- Величина шага $\left\{ \gamma_k = \frac{2}{k+2} \right\} _{k=0}$
- Стартовая точка $ x^0 \in \mathbb{R}^d $
- Количество итераций $ K $

*$k$-ая итерация:*
1. Подсчитать направление спуска:  
   $$ \nabla f(x^k) $$
   
2. Найти оптимальное направление:  
   $$ s^k = \arg \min_{s \in \mathcal{X}} \langle s, \nabla f(x^k) \rangle $$

3. Обновить значение:  
   $$ x^{k+1} = (1 - \gamma_k) x^k + \gamma_k s^k $$

Используйте предложенную функцию для реализации алгоритма и допишите недостающие фрагменты. После чего для проверки правильности загрузите функцию в [контест TBD](https://contest.yandex.ru/contest/66540/enter/)

In [10]:
# Ваше решение
import numpy as np
from tqdm.autonotebook import trange

def FrankWolfe(grad, criterion, gamma, x_0, A, b, eps, max_iter):
    # сил больше нет

    '''
       grad(x) - функция, которая считает градиент целевой функции;
       criterion(x) - функция, которая считает критерий;
       x_0 - начальная точка;
       eps - точность сходимости (обычно 1e-8);
       max_iter - количество итераций.
    '''

    errors = []

    x_k = np.copy(x_0)
    err_x_0 = criterion(x_k, A, b)
    errors.append(criterion(x_k, A, b) / err_x_0)
    for k in trange(max_iter):

        # your code
        g = grad(x_k, A, b)
        s_k = np.zeros_like(x_k)
        j = np.argmax(np.abs(g))
        s_k[j] = 1
        x_k = x_k * (1 - gamma) + s_k * gamma
        
        errors.append(criterion(x_k, A, b) / err_x_0)
        if errors[-1] < eps:
            break
            
    return x_k, errors

__в). (2 балла)__ Решите задачу оптимизации на обучающей выборке с помощью реализованного метода. Возьмите $R = 5$ и стартовую точку в $0$. В качестве критерия используйте следующее выражение:
$$
\text{gap}(w^k) = \max_{y \in \mathcal{X}} \langle \nabla f(w^k), w^k - y \rangle,
$$
и усредненную версию $\frac{1}{k} \sum_{i=1}^k \text{gap}(w^i)$. Такой критерий используем, так как не знаем значение $f^*$ и не можем гарантировать, что $\nabla f(w^*) = 0$. Но можно показать, что $\text{gap}(w^k) \geq f(w^k) - f^*$, а также доаказать сходимость метода Франк-Вульфа по такому критерию, а значит сходимость по $\text{gap}(w^k)$ и дает хорошее понимание о поведении $f(w^k) - f^*$.

Постройте два графика сходимости: значение критерия сходимости (две версии, обычная и усреднённая) от номера итерации.

In [ ]:
# Ваше решение

Выведите решение, полученное с помощью метода Франк-Вульфа. Что необычного увидели? Возможно, какое-то значение часто повторяется? Объясните, почему так получается.

In [ ]:
# Ваше решение

__Ваше решение__

### Дополнительная часть

__Задача 1. (всего 2 балла)__

В прошлом задании мы, используя полученное решения задачи оптимизации, предсказывали ответы на тестовой выборке. Напомним суть: исходная задача регрессии является задачой машинного обучения и с помощью линейной модели $g$ можно предсказывать значения меток $y$. Пусть у нас есть сэмпл $x_i$, ответ модели для этого сэмпла есть $g(w^*, x^i)$. Тогда предсказывающее правило можно сформулировать следующим довольно естественным образом:
$$
y_i =
\begin{cases}
1, & g(w^*, x^i) \geq 0,
\\
0, & g(w^*, x^i) < 0.
\end{cases}
$$
Cделав предсказания на тестовой выборке $X_{test}$, можно сравните результат с реальными метками $y_{test}$. Количество правильно угаданных меток есть точность/accuracy модели.

Посмотрите какую дает модель обученная с помощью метода Франк-Вульфа. Варьируйте $R = 5, 10, 20, 50, 100, 1000$. Постройте три графика:
1) точность итоговой модели от $R$,
2) количество ненулевых компонент в итоговом решении метода Франк-Вульфа от $R$,
3) точность от количества ненулевых компонент в итоговом решении.

In [ ]:
# Ваше решение

Сделайте вывод. В первую очередь обратите внимание на точки максимума или минимума.

__Ваше решение__

__Задача 2. (всего 3 балла)__

Нашу задачу можнно решать и с помощью метода градиентного спуска с евклидовой проекцией. Для этого нужно уметь делать проекцию на $\ell_1$-шар, поэтому нужно вывести ее ручками.

__а). (1 балл)__ 
Рассмотрим задачу поиска проекции на $\ell_{1}$-шар радиуса $R$ с центром в 0:
$$
    \arg\min\limits_{|y|_1 \leq R} \left[ \frac{1}{2} | x - y |^{2}_{2} \right].
$$

Используя одно из необходимых условий ККТ, покажите, что оптимальное решение можно найти как

\begin{align*}
    \Pi_{\mathcal{X}} (x) =
    \begin{cases}
        x, & \text{если}\ \| x \|_{1} \leq R; \\
        \text{sign}(x_{i})(|x_{i}| - \lambda)_{+}\ \forall i \in \overline{1, d}, & \text{если}\ \| x \|_{1} > R,
    \end{cases}
\end{align*}
где $\lambda$ - решение двойственной задачи. Распишите достаточные условия ККТ и составьте уравнение, из которого можно найти $\lambda$.

*Hint: если записать кружочек в группу, то мы Вам дадим подсказку где найти решение)*

__Ваше решение__

*Hint: Воспользуйтесь теоремой Каруша-Куна-Такера и запишите Лагранжиан*

In [ ]:
# Ваше решение

__б). (2 балла)__ 
Решите задачу оптимизации на обучающей выборке с помощью градиентного спуска с евклидовой проекцией. (Код градиентного спуска можете взять из первой прак дз)

Сравните на графиках сходимость градиентного спуска и метода Франк-Вульфа: 1) значение критерия от номера итерации, 2) значение критерия от времени. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)